# Цель работы

Есть обезличенный набор признаков товаров интернет- магазина. Требуется  разработать алгоритм, который предложит несколько вариантов наиболее похожих товаров. Целевая метрика -  accuracy@5 >=0,8 на валидационной выборке.

В этом ноутбуке приведены эксперименты с загрузкой признаков по батчам

# Импорт библиотек

In [ ]:
!apt install libomp-dev
!pip install faiss-gpu
!pip install catboost

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 6s (117 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 120901 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a14

Загрузка

In [ ]:

import pickle

import numpy as np
import pandas as pd
import joblib
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import StandardScaler

import os
import re
import pickle
import catboost
from catboost.core import CatBoostClassifier, sum_models

In [ ]:
import faiss

In [ ]:
data_train = pd.read_csv('drive/MyDrive/Colab Notebooks/matching/data/train.csv', index_col=0)
data_valid = pd.read_csv('drive/MyDrive/Colab Notebooks/matching/data/validation.csv', index_col=0)
data_answers = pd.read_csv('drive/MyDrive/Colab Notebooks/matching/data/validation_answer.csv', index_col=0)
data_base = pd.read_csv('drive/MyDrive/Colab Notebooks/matching/data/base.csv', index_col=0)


data = {
    'train': data_train,
    'valid': data_valid,
    'answer': data_answers,
    'common': data_base
}

In [ ]:
const_columns = ['70', '25', '21']
data_train = data_train.drop(columns=const_columns, axis=1)
data_valid = data_valid.drop(columns=const_columns, axis=1)
data_base = data_base.drop(columns=const_columns, axis=1)

In [ ]:
features_train = data_train.drop(columns=['Target'], axis=1)
features_valid = data_valid
target_train = data_train['Target']
features_base = data_base

scaler = StandardScaler()
scaler.fit(features_train)

features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
features_base = scaler.transform(features_base)

In [ ]:
common_index = {k: v for k, v in enumerate(data_base.index.to_list())}

with open('drive/MyDrive/Colab Notebooks/matching/data/common_index.pkl', 'wb') as f:
    pickle.dump(common_index, f)
#with open('common_index.pkl', 'rb') as f:
    #common_index = common_index.load(f)

In [ ]:
del data_base

In [ ]:
dims = features_base.shape[1]
NeiNum = 100
n_cells = 200
batch_size = int(features_train.shape[0] / 10)
step = 0

Функция для поиска соседей на GPU

In [ ]:
def faiss_gen_gpu(data, dimensions, batch_size, step, number):
  res = faiss.StandardGpuResources()
  index = faiss.IndexFlatL2(dimensions)
  gpu_index = faiss.index_cpu_to_gpu(res, 0, index)
  gpu_index.add(np.ascontiguousarray(features_base).astype('float32'))
  start = step * batch_size
  stop = start + batch_size
  dist, idx_found = \
  gpu_index.search(np.ascontiguousarray(data[start:stop]).astype('float32'), number)
  index_list = np.concatenate(idx_found).tolist()
  cnd = []
  for _ in index_list:
    cnd.append(features_base[_].tolist())
  return dist, idx_found, np.array(cnd)

Расчет целевой метрики

In [ ]:
def acc_at_n(targets: pd.Series, found_idx: np.array, nei: int, batch_size, step):
  label = []
  start = step * batch_size
  stop = start + batch_size
  targets = targets.tolist()[start:stop]
  result = 0
  for i in range(len(targets)):
    candidates = []
    for number in range(nei):
      candidates.append(common_index[found_idx[i][number]])
      if targets[i] == common_index[found_idx[i][number]]:
        label.append(1)
      else:
        label.append(0)
    if targets[i] in candidates:
      result += 1

  return result / len(targets), label

Сборка матрицы признаков

In [ ]:
def collect_data(distance, index, candidates, data, target, number, step, batch_size):
  dist = distance.reshape(-1, 1)
  start = step * batch_size
  stop = start + batch_size
  vectors = np.repeat(data[start:stop], number, axis=0)
  labels = np.array(target).reshape(-1, 1)
  cand_features = candidates
  cand_index = np.concatenate(index).reshape(-1, 1)
  return np.hstack((vectors, cand_features, dist, cand_index, labels))

Поиск соседей

In [ ]:
m_list = []
for step in range(0,10):
  distance, index, candidates = faiss_gen_gpu(features_train, dims, batch_size, step, NeiNum)
  accuracy, labels = acc_at_n(target_train, index, NeiNum, batch_size, step)
  preprocessed_data = collect_data(distance, index, candidates, features_train, labels,
                                   NeiNum, step, batch_size)
  X = preprocessed_data[:, 0:138]
  y = preprocessed_data[:, 140].astype('int')
  w1 = sum(y) / y.shape[0]
  w0 = (y.shape[0] - sum(y)) / y.shape[0]
  del candidates, index, distance, preprocessed_data
  params = {'learning_rate': 0.5,
        'depth': 2,
        'n_estimators': 150,
        'rsm': 1,
        'verbose': False,
        'loss_function': 'Logloss',
        'eval_metric': 'Accuracy',
        'class_weights': (w0, w1),
        'random_state': 2908,
        'task_type': 'GPU'
       }
  model = CatBoostClassifier(**params)
  model.fit(X, y)
  del X, y
  print('STEP---', step)
  m_list.append(model)


Суммирование предобученного бустинга для валидации

In [ ]:
model.save_model('drive/MyDrive/Colab Notebooks/matching/data/summ.cat')
new_model = CatBoostClassifier()
new_model.load_model('drive/MyDrive/Colab Notebooks/matching/data/summ.cat')

### Подготовка матрицы признаков для валидации

In [ ]:
def collect_data_v(distance, index, candidates, data, number, step, batch_size):
  start = step * batch_size
  stop = start + batch_size
  index = np.concatenate(index).reshape(-1, 1)
  distance = distance.reshape(-1, 1)
  vectors = np.repeat(data[start:stop], number, axis=0)
  cand_features = candidates
  return np.hstack((vectors, cand_features, distance, index))

In [ ]:
def acc_final(candidates, true_values, probs, number):
  result_100 = 0
  result_5 = 0
  answer = pd.DataFrame({'id':candidates.tolist(), 'probability':probs.tolist(), 'true_answer':true_values})
  for i in range(0, answer.shape[0], number):
    ids_5 = answer[i:i+number].sort_values(by='probability', ascending=False)[:5]['id']
    ids_100 = answer[i:i+number]['id']
    real_ans = answer[i:i+number]['true_answer'].values[0]
    if real_ans in [common_index[_] for _ in ids_100.values]:
      result_100 += 1
    if real_ans in [common_index[_] for _ in ids_5.values]:
      result_5 += 1
    size = answer.shape[0] / number

  return result_100 / size, result_5 / size

Расчет метрики на валидационной выборке

In [ ]:
p_list = []
for step in range(0,10):
  distance, index, candidates = faiss_gen_gpu(features_valid, dims, batch_size, step, NeiNum)
  accuracy, labels = acc_at_n(target_train, index, NeiNum, batch_size, step)
  preprocessed_data_v = collect_data_v(distance, index, candidates, features_valid, NeiNum, step, batch_size)
  X_v = preprocessed_data_v[:, 0:138]
  start = step * batch_size
  stop = start + batch_size
  ans = data_answers[start:stop]
  true_answer = pd.DataFrame(np.repeat(ans.values, NeiNum, axis=0))[0]
  candidates = (preprocessed_data_v[:, 139]).astype('int')
  del index, distance, preprocessed_data_v
  prob_pred_CC = new_model.predict_proba(X_v)[:,1]
  del X_v
  acc_100, acc_5 = acc_final(candidates, true_answer, prob_pred_CC, NeiNum)
  del candidates
  print('RESULTS FOR STEP---', step)
  print(f'ACCURACY @ {NeiNum} --- {acc_100}, ACCURACY @ 5 --- {acc_5},')
  p_list.append(prob_pred_CC)

RESULTS FOR STEP--- 0
ACCURACY @ 100 --- 0.7921, ACCURACY @ 5 --- 0.6771,
RESULTS FOR STEP--- 1
ACCURACY @ 100 --- 0.7908, ACCURACY @ 5 --- 0.671,
RESULTS FOR STEP--- 2
ACCURACY @ 100 --- 0.7984, ACCURACY @ 5 --- 0.6825,
RESULTS FOR STEP--- 3
ACCURACY @ 100 --- 0.7876, ACCURACY @ 5 --- 0.6755,
RESULTS FOR STEP--- 4
ACCURACY @ 100 --- 0.792, ACCURACY @ 5 --- 0.6756,
RESULTS FOR STEP--- 5
ACCURACY @ 100 --- 0.794, ACCURACY @ 5 --- 0.6838,
RESULTS FOR STEP--- 6
ACCURACY @ 100 --- 0.7943, ACCURACY @ 5 --- 0.6836,
RESULTS FOR STEP--- 7
ACCURACY @ 100 --- 0.7932, ACCURACY @ 5 --- 0.6779,
RESULTS FOR STEP--- 8
ACCURACY @ 100 --- 0.7933, ACCURACY @ 5 --- 0.6778,
RESULTS FOR STEP--- 9
ACCURACY @ 100 --- 0.7971, ACCURACY @ 5 --- 0.6801,
